In [1]:
import numpy as np
import pickle
from sklearn import decomposition
from scipy.stats import spearmanr
from functools import reduce
import matplotlib.ticker as ticker
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio import SeqIO
import matplotlib.colors as mcolors
import pylab as pl
from collections import Counter
from matplotlib import colors as colors
from snakemake import io as smio
from Bio import Alphabet
from Bio.Alphabet import Reduced
from Bio.Data import CodonTable
from scipy.stats import chi2_contingency
import math 

plt.style.use('default')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
sns.set_style("whitegrid")

mpl.rc('font', family='sans-serif', serif='Helvetica') 
sns.set(font="Helvetica")

mpl.rcParams['pdf.fonttype']= 42
# fig.patch.set_facecolor('white')
%matplotlib inline

In [2]:
%cd /qbio/junsoopablo/2022_bioinfo/project/binfo1-datapack1

/qbio/junsoopablo/2022_bioinfo/project/binfo1-datapack1


In [66]:
!grep -i mirlet7f-1 gencode.gtf

chr13	ENSEMBL	gene	48691305	48691393	.	-	.	gene_id "ENSMUSG00000105621.3"; gene_type "miRNA"; gene_name "Mirlet7f-1"; level 3; mgi_id "MGI:2676798";
chr13	ENSEMBL	transcript	48691305	48691393	.	-	.	gene_id "ENSMUSG00000105621.3"; transcript_id "ENSMUST00000198652.3"; gene_type "miRNA"; gene_name "Mirlet7f-1"; transcript_type "miRNA"; transcript_name "Mirlet7f-1-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676798"; tag "basic";
chr13	ENSEMBL	exon	48691305	48691393	.	-	.	gene_id "ENSMUSG00000105621.3"; transcript_id "ENSMUST00000198652.3"; gene_type "miRNA"; gene_name "Mirlet7f-1"; transcript_type "miRNA"; transcript_name "Mirlet7f-1-201"; exon_number 1; exon_id "ENSMUSE00001348852.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676798"; tag "basic";


In [67]:
!samtools view -b -o CLIP-let7f-1.bam CLIP-35L33G.bam chr13:48691305-48691393
!samtools view CLIP-let7f-1.bam | wc -l

158


In [68]:
!samtools mpileup CLIP-let7f-1.bam > CLIP-let7f-1.pileup
!wc -l CLIP-let7f-1.pileup

[mpileup] 1 samples in 1 input files
97 CLIP-let7f-1.pileup


In [69]:
!head CLIP-let7f-1.pileup

chr13	48691290	N	1	^Gg	A
chr13	48691291	N	26	c^Hc^Ic^Dc^Hc^Gc^Ic^Gc^Ic^Gc^Ic^Ic^Hc^Hc^Hc^Ec^Ic^Ic^Ic^Hc^Hc^Hc^Ic^Ic^Ic^Hc	DDI4G>I@H;GGGFH7IHIHHHFIIG
chr13	48691292	N	25	aaaaaaaaaaaaaaaaaaaaaaaaa	DGIG>IBI?IIGHH7HIEHFFGHIG
chr13	48691293	N	38	gggggggggggggggggggggggggg^Ig^Hg^Ig^Ig^Ig^Ig^Hg^Ig^Ig^Dg^Ig^Ig	DGI4>AIGIBIIGGE/IIIDGHIIIHEGIHIIBIIBII
chr13	48691294	N	49	cccccccccccccccccccccccccccccccccccccc^Ec^Hc^Ic^Hc^Hc^Gc^Ic^Ic^Gc^Ic^Hc	?>I1E5IDG<I>EHH7IIIFHHEIHGCEDIHIEHG?II3BIDD7IHGGH
chr13	48691295	N	58	aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa^Ha^Ia^Ha^Ha^Ha^Ia^Ia^Ia^Ha	BDI?HBIGHGIHGHG7IIIGBGIIIGBEFIIIBIH=II=EIGA6IHGIHDIHGGIII0
chr13	48691296	N	81	aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa^Ia^Ia^Da^Ia^Ha^Ia^Ha^Ga^Ha^Ha^Ga^Ia^Ha^Ha^Ia^Ha^Ha^Ia^Ha^Ia^Ga^Ia^Ga	<GI7H?IGIEIIGHG;FIIGGHIIIGIBGDIHGII@II9FHDA9IIGGHGIEGGIIIBEIBIHIG;DG?IBHICGIHI6I;
chr13	48691297	N	80	gggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg	8EI.HBIGIGIGGHG1IIIGGGIIIEIB

In [70]:
!awk '$2 >= 48691305 && $2 <= 48691393 { print $0; }' CLIP-let7f-1.pileup > CLIP-let7f-1-gene.pileup
!tail CLIP-let7f-1-gene.pileup

chr13	48691377	N	24	tttttttttttttttttttttttt	BIIGIGEDCGFIFGI@GHIDDIGI
chr13	48691378	N	24	aaaaaaaaaaaaaaaaaaaaaaaa	?HIFIGGGAFEFABI>GII;EIGI
chr13	48691379	N	24	cccccccccccccccccccccccc	EIIGIGGDEGGIEGI;GIIDDIGI
chr13	48691380	N	24	tttttttttttttttttttttttt	EIIGIBGGEGFIFGI@GIIBDIGI
chr13	48691381	N	24	aaaaaaaaaaaaaaaaaaaaaaaa	BEHFIGEG@FCEAGH4GII;AIDH
chr13	48691382	N	24	cccccccccccccccccccccccc	?IEGIHGG?GEGEBI=GIIDEIGG
chr13	48691383	N	24	cccccccccccccccccccccccc	EIGGIGGGAGFGFGI=GIIDGIGG
chr13	48691384	N	24	tttttttttttttttttttttttt	<IGGIHGG>GEGBGI8GII@GIGG
chr13	48691385	N	24	cccccccccccccccccccccccc	BIGGIBDD?GEGBEI=GIIBGIGG
chr13	48691386	N	24	a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$	EIGGIFDECGFGFGI2BIIDGIGG


In [71]:
import pandas as pd

pileup = pd.read_csv('CLIP-let7f-1-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

,chrom,pos,_ref,count,basereads,quals
77,chr13,48691382,N,24,cccccccccccccccccccccccc,?IEGIHGG?GEGEBI=GIIDEIGG
78,chr13,48691383,N,24,cccccccccccccccccccccccc,EIGGIGGGAGFGFGI=GIIDGIGG
79,chr13,48691384,N,24,tttttttttttttttttttttttt,<IGGIHGG>GEGBGI8GII@GIGG
80,chr13,48691385,N,24,cccccccccccccccccccccccc,BIGGIBDD?GEGBEI=GIIBGIGG
81,chr13,48691386,N,24,a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$,EIGGIFDECGFGFGI2BIIDGIGG


In [72]:
import re
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [73]:
pileup[['chrom', 'pos', 'matches']]

,chrom,pos,matches
0,chr13,48691305,cccccccccccccccccccccccccccccccccccccccccccccc...
1,chr13,48691306,tttttttttttttttttttttttttttttttttttttttttttttt...
2,chr13,48691307,cccccccccccccccccccccccccccccccccccccccccccccc...
3,chr13,48691308,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
4,chr13,48691309,gggggggggggggggggggggggggggggggggggggggggggggg...
...,...,...,...
77,chr13,48691382,cccccccccccccccccccccccc
78,chr13,48691383,cccccccccccccccccccccccc
79,chr13,48691384,tttttttttttttttttttttttt
80,chr13,48691385,cccccccccccccccccccccccc


In [74]:
pileup[pileup['pos'] == 48691305].iloc[0]['matches']

'ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc'

In [75]:
pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr13,48691305,N,109,cccccccccccccccccccccccccccccccccccccccccccccc...,B?IBHBIGI>GGHHH:IIHFGGIIIGHHIHIIBIG?IGBFIE>@II...,cccccccccccccccccccccccccccccccccccccccccccccc...
1,chr13,48691306,N,109,tttttttttttttttttttttttttttttttttttttttttttttt...,=GH0H=IBI>IHHGH?IIIGGHBII@HHIGHIDIIIGDHIFGEIHG...,tttttttttttttttttttttttttttttttttttttttttttttt...
2,chr13,48691307,N,109,cccccccccccccccccccccccccccccccccccccccccccccc...,;GIBH=IGIAEHH:H:IIIHEHIIIEIDHHIIGII:IIDBI<>GII...,cccccccccccccccccccccccccccccccccccccccccccccc...
3,chr13,48691308,N,109,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,AGI?H6IGI?IHHHHBIIIHEHHIIGIHIIIIGII<IIDFIEE<II...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
4,chr13,48691309,N,109,gggggggggggggggggggggggggggggggggggggggggggggg...,>EI>F=IDGEIIHHGBIIIHHHIIIGIHHHIIDII:II5HIGG@II...,gggggggggggggggggggggggggggggggggggggggggggggg...
...,...,...,...,...,...,...,...
77,chr13,48691382,N,24,cccccccccccccccccccccccc,?IEGIHGG?GEGEBI=GIIDEIGG,cccccccccccccccccccccccc
78,chr13,48691383,N,24,cccccccccccccccccccccccc,EIGGIGGGAGFGFGI=GIIDGIGG,cccccccccccccccccccccccc
79,chr13,48691384,N,24,tttttttttttttttttttttttt,<IGGIHGG>GEGBGI8GII@GIGG,tttttttttttttttttttttttt
80,chr13,48691385,N,24,cccccccccccccccccccccccc,BIGGIBDD?GEGBEI=GIIBGIGG,cccccccccccccccccccccccc


In [76]:
from scipy.stats import entropy
from collections import Counter

def entropy1(labels, base=None):
  value,counts = np.unique(labels, return_counts=True)
  return round(entropy(counts, base=base), 3)

def entropy2(matches):
    basedic = Counter(list(matches))
    total = sum(basedic.values())
    return entropy1([i/total for i in basedic.values()])

In [77]:
pileup['match_count'] = pileup.apply(lambda x: len(x['matches']), axis=1)
pileup['entropy'] = pileup.apply(lambda x: entropy2(x['matches']), axis=1)
pileup.head(20)

,chrom,pos,_ref,count,basereads,quals,matches,match_count,entropy
0,chr13,48691305,N,109,cccccccccccccccccccccccccccccccccccccccccccccc...,B?IBHBIGI>GGHHH:IIHFGGIIIGHHIHIIBIG?IGBFIE>@II...,cccccccccccccccccccccccccccccccccccccccccccccc...,109,0.000
1,chr13,48691306,N,109,tttttttttttttttttttttttttttttttttttttttttttttt...,=GH0H=IBI>IHHGH?IIIGGHBII@HHIGHIDIIIGDHIFGEIHG...,tttttttttttttttttttttttttttttttttttttttttttttt...,109,0.000
2,chr13,48691307,N,109,cccccccccccccccccccccccccccccccccccccccccccccc...,;GIBH=IGIAEHH:H:IIIHEHIIIEIDHHIIGII:IIDBI<>GII...,cccccccccccccccccccccccccccccccccccccccccccccc...,109,0.000
3,chr13,48691308,N,109,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,AGI?H6IGI?IHHHHBIIIHEHHIIGIHIIIIGII<IIDFIEE<II...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,109,0.000
4,chr13,48691309,N,109,gggggggggggggggggggggggggggggggggggggggggggggg...,>EI>F=IDGEIIHHGBIIIHHHIIIGIHHHIIDII:II5HIGG@II...,gggggggggggggggggggggggggggggggggggggggggggggg...,109,0.000
5,chr13,48691310,N,116,gggggggggggggggggggggggggggggggggggggggggggggg...,<GI7D@I?IFIIHHH?IIIHGHIIIBIGIIIIFGH5II5DIBE@II...,gggggggggggggggggggggggggggggggggggggggggggggg...,123,1.040
6,chr13,48691311,N,117,gggggggggggggggggggggggggggggggggggggggggggggg...,@HI7DBIGI<FIHHH?IIIHDHIIIHI@IIIIGHI>HI0HIGG;II...,gggggggggggggggggggggggggggggggggggggggggggggg...,117,0.000
7,chr13,48691312,N,132,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,GHI;E?IFIFIIHHH@IIIHGHIIIDIGIIIIGIIBII7FIGG@II...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,147,1.609
8,chr13,48691313,N,132,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,GHI;DBIEI8IIGGH:IIIHGFIIIGIEIIIIGHIBHI;HIGG;II...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,132,0.000
9,chr13,48691314,N,131,gggggggggggggggggggggggggggggggggggggggggggggg...,CHI9HEIDIGIIHHH?IIIGGFHIIGIGIIIIGIIBII6GIBG@II...,gggggggggggggggggggggggggggggggggggggggggggggg...,131,0.000


In [78]:
pileup_bed = pileup[['chrom','pos','pos','entropy']]
pileup_bed.columns = ['chrom','start','end','entroypy']
pileup_bed['start'] = pileup_bed['start'] -1
pileup_bed.head()

/tmp/ipykernel_2011491/2736039230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pileup_bed['start'] = pileup_bed['start'] -1


,chrom,start,end,entroypy
0,chr13,48691304,48691305,0.0
1,chr13,48691305,48691306,0.0
2,chr13,48691306,48691307,0.0
3,chr13,48691307,48691308,0.0
4,chr13,48691308,48691309,0.0


In [79]:
pileup_bed.to_csv('CLIP-let7f-1-gene_entropy.bed', sep='\t', index=False, header=False)